# IGROdisha Plot Value Scraper

Quick and dirty portable Jupyter implementation. 

## Importing libraries and assigning directory

In [11]:
import os, winsound
import pandas as pd
import requests, time

In [ ]:
base_dir = r""

## Classes

<i>progressbar</i> builds basic progressbar for multiple functions.
<i>timecheck</i> adds time calculator functionality.

In [13]:
class progressbar():
    '''Default progress bar used for multiple functions.'''
    def dispBar(iter, constr,  res):
        pct = 100 * (iter/constr)
        blno = int((pct/100) * res)
        bl_str = "▒" * blno
        if iter != constr: line_str = "_" * (res-blno)
        else: line_str = ""
        print(bl_str+line_str, "%.3f percent done..." %pct, end='\r')


In [14]:
class timecheck:
    '''Time calculator functionality.'''
    tms = []

    def constrfind(id):
        '''Finds total plots under given registry office.'''
        global RestrtCh
        ss = time.time()
        vil_payl = {'RegoffId':id}
        vil_json = s.post(vil_api,json=vil_payl).json()
        pts = 0; i = 0
        for vil in vil_json['d']:
            plot_payl = {'StrVillageId':vil['VILL_ID']}
            plot_json = s.post(plot_api, json=plot_payl).json()
            pts = pts + len(plot_json['d'])
            i+=1
            # print("\rChecking for timecheck constraint: %.3f percent completed..." %((i/len(vil_json['d']))*100), end='\r')
            progressbar.dispBar(i, len(vil_json['d']), 50)
        timecheck.constr = pts
        # if RestrtCh=='y':timecheck.Recalc()
        ff = time.time()
        print("\nTotal %i plots to evaluate." %int(pts))
        print("Calculation took {} mins, {.4f} seconds.".format(*divmod(ff-ss,60)))

    def Recalc():
        '''Recalculates plots remaining based on files added to directory.'''
        global DistId, RegName
        dir = os.path.join(base_dir,str(DistId),RegName)
        done = len(list(os.scandir(dir)))
        timecheck.constr = timecheck.constr - done

    def estTimeRemain(start_tm, iter):
        '''Estimates time remaining based on last 400 iterations.'''
        tm = time.time() - st
        timecheck.tms.append(tm)
        avg = sum(timecheck.tms)/len(timecheck.tms)
        if len(timecheck.tms)>500:avg = sum(timecheck.tms[-400:])/401
        rem = timecheck.constr - iter
        est = avg*rem
        min_est = int(est//60)
        hrs, mins = divmod(min_est, 60)
        if len(timecheck.tms)>100 and len(timecheck.tms)%100==0:print("Estimated time remaining: {}hrs {}mins...".format(int(hrs), int(mins)))
        if len(timecheck.tms)%400==0:print("Average time per iteration: %.5f seconds." %avg)


## Functions

<i>findRegOffs</i> finds and displays registration offices in a district. <i>retVill</i> counts number of plots downloaded based on files saved in dedicated fileorder in working directory. <i>initlists</i> initializes lists.

In [15]:
def findRegOffs(dist):
    '''Finds and displays registry offices in a particular district.'''
    global RegId, RegName
    i = 0
    payl = {"distId":dist}
    json=s.post(regoff_api,json=payl).json()
    print("There are", len(json['d']), "registration offices in this district.")
    # print(json['d'][2]['REGOFF_ID'])
    # df.to_csv("E:/Research/testjson.csv")
    for regs in json['d']:
        print(str(i)+".", "\t\t",  regs['REGOFF_ID'], "\t\t", regs['REGOFF_NAME']); i+=1
    iselect = int(input("Select Registration Office to scrape: "))
    RegId = json['d'][iselect]['REGOFF_ID']
    RegName = json['d'][iselect]['REGOFF_NAME']
    print (RegName, RegId)


In [16]:
def retVill(d_id, regname):
    '''Counts number of plots assessed based on number of files saved to directory.'''
    global fctr
    dir = os.path.join(base_dir,str(d_id),regname)
    done = len(list(os.scandir(dir)))
    fctr = done
    return done*50


In [17]:
def initLists():
    global dists, regoffs, vils, plots 
    global kisms, mrvals, dts, txndts 
    dists, regoffs, vils, plots = [], [], [], []
    kisms, mrvals, dts, txndts = [], [], [], []


## Building Requests module and API addresses

In [18]:
s = requests.session()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
regoff_api = "https://www.igrodisha.gov.in/ViewFeeValue.aspx/GetRegoffice"
vil_api = "https://www.igrodisha.gov.in/ViewFeeValue.aspx/GetVillage"
plot_api = "https://www.igrodisha.gov.in/ViewFeeValue.aspx/GetPlotDtl"
ksmplt_api = "https://www.igrodisha.gov.in/ViewFeeValue.aspx/GetKismByPlot"
mrval_api = "https://www.igrodisha.gov.in/ViewFeeValue.aspx/GetMRVal"

## Main code body

In [19]:
initLists()
ctr, fctr  = 0, 0

In [20]:
DistId = input("Enter district ID:\t")
# DistId = 10
findRegOffs(DistId)

Enter district ID:	2
There are 10 registration offices in this district.
0. 		 6 		 BALASORE
1. 		 209 		 BALASORE ADM
2. 		 7 		 BALIAPAL
3. 		 8 		 BASTA
4. 		 9 		 JALESWAR
5. 		 10 		 JALESWARPUR
6. 		 11 		 KHAIRA
7. 		 12 		 NILAGIRI
8. 		 13 		 SIMULA
9. 		 14 		 SORO
Select Registration Office to scrape: 0
BALASORE 6


In [ ]:
RestrtCh = input("Restart from last savepoint? Enter 'y' or 'n'. ")
if RestrtCh=='n':timecheck.constrfind(RegId)
if RestrtCh=='y':lim = retVill(DistId, RegName)

In [ ]:
vil_payl = {'RegoffId':RegId}
vil_json = s.post(vil_api,json=vil_payl).json()
vctr = 0
for vil in vil_json['d']:
    vctr+=1
    if ctr==0 and RestrtCh=='y':        #breaker for resumefxn
        if vctr<=lim:
            progressbar.dispBar(vctr, lim, 30)
            continue
    print("\nReading data for", vil['VILL_ID'], "village under", RegName, "\b." )
    print("%i villages left to read." %int(len(vil_json['d'])-vctr))
    plot_payl = {'StrVillageId':vil['VILL_ID']}
    plot_resp = s.post(plot_api, json=plot_payl)
    if plot_resp.status_code == 200:
        try:
            plot_json = plot_resp.json()
        except:
            print("API lookup succeeded but data not readable for village", vil['VILL_NAME'], end=".\n")
            continue
    else:
        print("Unable to read data for village", vil['VILL_NAME'], "from", RegName, end=".\n")
        continue
    # plot.json()
    pctr = 0
    for plot in plot_json['d']:
        st = time.time()
        ctr+=1; pctr+=1
        ksmplt_payl = {'plotId':plot['One'], 'VillageId':vil['VILL_ID']}
        try:
            ksmplt_json = s.post(ksmplt_api, json=ksmplt_payl).json()
            ksm = ksmplt_json['d'][0]['PLOTCAT_TYPE']
            if len(ksmplt_json['d'])>1:
                ksm = "Multiple kisams found"
        except:
            ksm = "No Kisam found"
        mrval_payl = {'Dist':DistId, 'RegoffId':RegId, 'village':vil['VILL_ID'], 'Plot':plot['One'], 'Area':'1', 'Unit': '1', 'unitTest':'Acre'}
        try:
            mrval_json = s.post(mrval_api, json=mrval_payl).json()
            mrval = mrval_json['d'].split("@$")[1]
        except:
            mrval = "error"
            dt = "error"
            txndt = "error"
        else:
            try:
                dt = mrval_json['d'].split("@$")[6]
                txndt = mrval_json['d'].split("@$")[7]
            except:
                dt = "error"
                txndt = "error"
        dists.append(DistId)
        regoffs.append(RegId)
        vils.append(vil['VILL_ID'])
        plots.append(plot['One'])
        kisms.append(ksm)
        mrvals.append(mrval)
        dts.append(dt)
        txndts.append(txndt)
        if pctr%10==0:print("Read", pctr, "of", len(plot_json['d']), "values.", end='\r')
        # if RestrtCh=='n':timecheck.estTimeRemain(st, ctr)
    else:    
        if vctr%50==0:       
            ss = time.time()
            df = pd.DataFrame({'District':dists, 'RegistrationOff':regoffs, 'Village':vils, 'PlotID':plots, 'Kisam':kisms, 'Value':mrvals, 'Date':dts, 'TransactionDate':txndts})
            loc = os.path.join(base_dir,str(DistId),RegName)
            if not os.path.exists(loc):os.makedirs(loc)
            df.to_csv(os.path.join(loc,"{}_{}.csv".format(RegName, fctr+1))); fctr+=1
            ff = time.time()
            print("\nSaving directory to", loc, "took %.4f seconds." %(ff-ss), end=".\n")
            initLists()
df = pd.DataFrame({'District':dists, 'RegistrationOff':regoffs, 'Village':vils, 'PlotID':plots, 'Kisam':kisms, 'Value':mrvals, 'Date':dts, 'TransactionDate':txndts})
loc = os.path.join(base_dir,str(DistId),RegName)
if not os.path.exists(loc):os.makedirs(loc)
df.to_csv(os.path.join(loc,"{}_rem.csv".format(RegName)), mode='a')
print("Saved directory to", loc, end=".\n")
for i in range(8):winsound.Beep(550, 750)       #alert sound